In [1]:
import numpy as np
import json
import os

import subprocess

In [2]:
%matplotlib inline
%config Completer.use_jedi = False

In [3]:
import subprocess

useless_cat_call = subprocess.run(["ls", "-lh"], stdout=subprocess.PIPE)
print(useless_cat_call.stdout)  # Hello from the other side

b'total 16\ndrwxr-xr-x  35 sergmiller  staff   1.1K Sep 30 20:39 errorlogs\ndrwxr-xr-x  48 sergmiller  staff   1.5K Sep 30 20:39 replays\n-rw-r--r--   1 sergmiller  staff   7.4K Sep 30 20:40 research-v1.ipynb\n'


In [4]:
useless_cat_call.returncode

0

In [118]:
simple_bot = "../submissions/simple/main.py"
replays = "replays"

def run_game(left_bot=simple_bot, right_bot=simple_bot, seed=42):
    replay_path = "replay.json"
    python_v = "python3.7"
    
    replay_path = os.path.join(replays, str(np.random.randint(1e9)) + ".json")
    
    res = subprocess.run([
        "lux-ai-2021",
        left_bot,
        right_bot,
#         "--statefulReplay",
        "--python={}".format(python_v),
        "--seed={}".format(seed),
        "--out={}".format(replay_path)], stdout=subprocess.PIPE)
    
    print(res.stdout.decode())

    assert res.returncode == 0

    with open(replay_path, "r") as f:
        result = json.load(f)
    return result

In [149]:
zero_bot = "../submissions/zero_actions/main.py"
r = run_game(simple_bot, simple_bot, 42)


-=-=-=-=-=-=-=-=-=-=-=-| [INFO] match_RyUefHP8r81M |-=-=-=-=-=-=-=-=-=-=-=-

[INFO] (match_RyUefHP8r81M) - Design: lux_ai_2021 | Initializing match - ID: RyUefHP8r81M, Name: match_RyUefHP8r81M
[WARN] (match_RyUefHP8r81M) - turn 29; Unit u_2 collided when trying to move w to (13, 10)
[WARN] (match_RyUefHP8r81M) - turn 42; Unit u_8 collided when trying to move s to (0, 11)
[WARN] (match_RyUefHP8r81M) - turn 43; Unit u_6 collided when trying to move n to (13, 10)
[WARN] (match_RyUefHP8r81M) - turn 43; Unit u_9 collided when trying to move w to (13, 10)
[WARN] (match_RyUefHP8r81M) - turn 45; Unit u_8 collided when trying to move s to (0, 12)
[WARN] (match_RyUefHP8r81M) - turn 50; Unit u_8 collided when trying to move w to (0, 10)
[WARN] (match_RyUefHP8r81M) - turn 51; Unit u_8 collided when trying to move w to (0, 10)
[WARN] (match_RyUefHP8r81M) - turn 51; Unit u_4 collided when trying to move w to (1, 10)
[WARN] (match_RyUefHP8r81M) - turn 52; Unit u_8 collided when trying to move w to (

In [109]:
r.keys()

dict_keys(['seed', 'allCommands', 'mapType', 'teamDetails', 'version', 'stateful', 'results'])

In [110]:
r["version"]

'3.1.0'

In [111]:
r["results"]

{'ranks': [{'rank': 1, 'agentID': 0}, {'rank': 2, 'agentID': 1}],
 'replayFile': 'replays/615856404.json'}

In [112]:
r['stateful']

[{'turn': 0,
  'globalCityIDCount': 2,
  'globalUnitIDCount': 2,
  'teamStates': {'0': {'researchPoints': 0,
    'units': {'u_1': {'cargo': {'wood': 0, 'coal': 0, 'uranium': 0},
      'cooldown': 0,
      'x': 1,
      'y': 11,
      'type': 0}},
    'researched': {'wood': True, 'coal': False, 'uranium': False}},
   '1': {'researchPoints': 0,
    'units': {'u_2': {'cargo': {'wood': 0, 'coal': 0, 'uranium': 0},
      'cooldown': 0,
      'x': 14,
      'y': 11,
      'type': 0}},
    'researched': {'wood': True, 'coal': False, 'uranium': False}}},
  'map': [[{'road': 0, 'resource': {'type': 'coal', 'amount': 352}},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0, 'resource': {'type': 'coal', 'amount': 352}}],
   [{'road': 0},
    {'road': 0, 'resource': {'type': 'coal', 'amount': 399}}

In [12]:
def run_series(left_bot, right_bot, seed=42, n_runs=10):
    results = []
    for t in range(n_runs):
        results.append(run_game(left_bot, right_bot, seed + t))
    return results

In [14]:
%%time

r = run_series(simple_bot, zero_bot)

b"\n-=-=-=-=-=-=-=-=-=-=-=-| \x1b[INFO]\x1b \x1b\x1bmatch_BNIbT1tf9vhy\x1b\x1b |-=-=-=-=-=-=-=-=-=-=-=-\n\n\x1b[INFO]\x1b (match_BNIbT1tf9vhy) - Design: lux_ai_2021 | Initializing match - ID: BNIbT1tf9vhy, Name: match_BNIbT1tf9vhy\n{\n  ranks: [\n    { rank: 1, agentID: 0, name: '../submissions/simple/main.py' },\n    {\n      rank: 2,\n      agentID: 1,\n      name: '../submissions/zero_actions/main.py'\n    }\n  ],\n  replayFile: 'replays/325426074.json',\n  seed: 42\n}\n"
b"\n-=-=-=-=-=-=-=-=-=-=-=-| \x1b[INFO]\x1b \x1b\x1bmatch_Fcm0X4tEiB1w\x1b\x1b |-=-=-=-=-=-=-=-=-=-=-=-\n\n\x1b[INFO]\x1b (match_Fcm0X4tEiB1w) - Design: lux_ai_2021 | Initializing match - ID: Fcm0X4tEiB1w, Name: match_Fcm0X4tEiB1w\n{\n  ranks: [\n    { rank: 1, agentID: 0, name: '../submissions/simple/main.py' },\n    {\n      rank: 1,\n      agentID: 1,\n      name: '../submissions/zero_actions/main.py'\n    }\n  ],\n  replayFile: 'replays/330403683.json',\n  seed: 43\n}\n"
b"\n-=-=-=-=-=-=-=-=-=-=-=-| \x1b[INFO]\

In [15]:
len(r)

10

In [16]:
r[0]['results']

{'ranks': [{'rank': 1, 'agentID': 0}, {'rank': 2, 'agentID': 1}],
 'replayFile': 'replays/325426074.json'}

In [19]:
r[0].keys()

dict_keys(['seed', 'allCommands', 'mapType', 'teamDetails', 'version', 'stateful', 'results'])

In [22]:
r[0]['stateful']

[{'turn': 0,
  'globalCityIDCount': 2,
  'globalUnitIDCount': 2,
  'teamStates': {'0': {'researchPoints': 0,
    'units': {'u_1': {'cargo': {'wood': 0, 'coal': 0, 'uranium': 0},
      'cooldown': 0,
      'x': 1,
      'y': 11,
      'type': 0}},
    'researched': {'wood': True, 'coal': False, 'uranium': False}},
   '1': {'researchPoints': 0,
    'units': {'u_2': {'cargo': {'wood': 0, 'coal': 0, 'uranium': 0},
      'cooldown': 0,
      'x': 14,
      'y': 11,
      'type': 0}},
    'researched': {'wood': True, 'coal': False, 'uranium': False}}},
  'map': [[{'road': 0, 'resource': {'type': 'coal', 'amount': 352}},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0},
    {'road': 0, 'resource': {'type': 'coal', 'amount': 352}}],
   [{'road': 0},
    {'road': 0, 'resource': {'type': 'coal', 'amount': 399}}

In [35]:
from subprocess import Popen, PIPE

session = subprocess.Popen(['ls', '-lh'] + '| grep 1'.split(), stdout=PIPE, stderr=PIPE)
stdout, stderr = session.communicate()

In [36]:
stdout, stderr

(b'',
 b'ls: 1: No such file or directory\nls: grep: No such file or directory\nls: |: No such file or directory\n')

In [ ]:
def run_game(agentA: str, agentB: str):
    os.